In [1]:
import threading
from typing import Tuple

import numpy as np
import plotly.graph_objects as go
from scipy.optimize import differential_evolution
import ipywidgets as w

# SI units for all quantities

# Regulation backboard
BACKBOARD_WIDTH = 1.83  
BACKBOARD_HEIGHT = 1.22

BASKET_Z_REL = 0.305
BASKET_Z = 3.05

BACKBOARD_Z = BASKET_Z - BASKET_Z_REL + BACKBOARD_HEIGHT / 2
BACKBOARD_Y = 4.572  # Assume all shots come from x=0 at the free throw line

BASKET_Y_REL = 0.23

N_GRID_POINTS = 21
HOOP_RADIUS = 0.4572 / 2
BASKET_Y = BACKBOARD_Y - BASKET_Y_REL - HOOP_RADIUS

G = 9.8

Z_VEL_SPACE = 0.04
N_Z_VEL = 30
COEFF_RESTITUTION = 0.5

xs = np.linspace(-BACKBOARD_WIDTH / 2, BACKBOARD_WIDTH / 2, N_GRID_POINTS)
ys = np.linspace(-BACKBOARD_HEIGHT / 2, BACKBOARD_HEIGHT / 2, N_GRID_POINTS)
xs, ys = np.meshgrid(xs, ys)
initial_backboard_points = np.array([xs, ys]).T.reshape(-1, 2)

MAX_X_POW = 7
MAX_Y_POW = 5 # really 2x max y power


def make_backboard_poly(a: np.array, points: np.array) -> np.array:
    """Model the backboard as a 2d polynomial surface
       with coefficients given by a. Due to symmetry, 
       only do even powers of y.
    """
    a = a.reshape((MAX_X_POW, MAX_Y_POW))
    xs = points[:, 1]
    ys = points[:, 0]
    s = np.zeros(len(points))
    for j in range(MAX_Y_POW):
        for i in range(MAX_X_POW):
            expo = 2 * j
            r = a[i, j] * xs**i * ys**expo
            s += r
    return s


def make_backboard_poly_grad(a, points):
    """Compute the gradient of the backboard polynomial"""
    a = a.reshape((MAX_X_POW, MAX_Y_POW))
    xs = points[:, 1]
    ys = points[:, 0]
    sx = np.zeros(len(points))
    for j in range(MAX_Y_POW):
        for i in range(MAX_X_POW):
            e = 2 * j
            if i == 0:
                r = 0
            else:
                r = i * a[i, j] * xs ** (i - 1) * ys**e
            sx += r

    sy = np.zeros(len(points))
    for j in range(MAX_Y_POW):
        expo = 2 * j
        for i in range(MAX_X_POW):
            if j == 0:
                r = 0
            else:
                r = expo * a[i, j] * xs**i * ys ** (expo - 1)
            sy += r

    return sy, sx



def setup_backboard(a: np.array, ps: np.array) -> Tuple[np.array, np.array]:
    """Create backboard points and normals, and rotate and translate to 
       correct positions
    """
    zs = make_backboard_poly(a, ps)
    points = np.array([ps[:, 0], ps[:, 1], zs]).T

    x, y = make_backboard_poly_grad(a, ps)
    # If f(x, y) = z, then normal is grad(f(x, y) - z)
    gs = np.array([x, y, -np.ones_like(x)]).T
    no = np.linalg.norm(gs, axis=1)
    gs = gs / no[:, None]

    gs = np.array([gs[:, 0], -gs[:, 2], gs[:, 1]]).T
    points = np.array([points[:, 0], -points[:, 2] + BACKBOARD_Y, points[:, 1] + BACKBOARD_Z]).T

    return points, gs


def resolve_collison(vels, norms):
    """Treating basketball as point particle, simulate collision 
       with backboard.
    """
    para = (vels * norms).sum(axis=1)[:, None] * norms
    mperp = np.cross(vels, norms)
    perp = np.cross(mperp, norms)
    new = -para * COEFF_RESTITUTION - perp
    return new




def compute_velocities(end_points: np.array, z_inc: int) -> np.array:
    """Given an end point, compute the starting velocity needed to reach such point.
       we can parameterize the set of arcing trajectories that go through a point
       by their z velocity. The z inc says how much to increment the z velocity 
       above the minimum velocity to hit the end point.
    """
    x0 = end_points[:, 0]
    y0 = end_points[:, 1]
    z0 = end_points[:, 2]
    a = 0.5 * G
    c = z0
    # Minimum is when discriminant is zero
    z_vel_min = 2 * (a * c) ** 0.5 + 1e-8
    z_vels = z_vel_min + z_inc * Z_VEL_SPACE + 1e-9
    b = -z_vels
    # Quadratic formula, take larger t as we only consider arcing shots
    t = (-b + (b**2 - 4 * a * c) ** 0.5) / (2 * a)
    y_vels = y0 / t
    x_vels = x0 / t
    return np.array([x_vels, y_vels, z_vels]).T, t


def hoop_distance(starts: np.array, vels: np.array) -> np.array:
    """Compute the distance from the center of the hoop the 
       ball is when it crosses the basket height. If it never
       gets to the basket height, return 1.
    """
    velz = vels[:, 2]
    c = BASKET_Z - starts[:, 2]
    b = -velz
    a = 0.5 * G
    disc = b**2 - 4 * a * c
    zs = disc < 0
    disc[zs] = 0

    t = (-b + disc**0.5) / (2 * a)
    t_neg = t < 0
    x = starts[:, 0] + vels[:, 0] * t
    y = starts[:, 1] + vels[:, 1] * t

    hoop_d = ((y - BASKET_Y) ** 2 + x**2) ** 0.5
    hoop_d[zs] = 1
    hoop_d[t_neg] = 1
    return hoop_d, t


def compute_baskets(coeffs: np.array) -> float:
    """For each grid point in a candidate backboard, consider 
       a shot which hits that point, and count up how many baskets
       go in. Return the percentage of shots that go in.
    """
    backboard_points, backboard_normals = setup_backboard(coeffs, initial_backboard_points)
    baskets = 0
    for zi in range(N_Z_VEL):
        # Compute what the starting velocity must be
        vel, t = compute_velocities(backboard_points, zi)
        # See what the velocity is when it hits the backboard
        vel[:, 2] = vel[:, 2] - G * t
        # Work out the velocity after collison
        col_vel = resolve_collison(vel, backboard_normals)
        # See how far away it is from hoop
        d, _ = hoop_distance(backboard_points, col_vel)
        # Total up the successful shots
        baskets += (d < HOOP_RADIUS).sum()

    # -ve to work with minimize functions
    return -baskets / (N_Z_VEL * len(backboard_points))


# %load_ext line_profiler
# %lprun -f compute_baskets compute_baskets(x0)

fig = go.FigureWidget()
fig.add_mesh3d(opacity=1, delaunayaxis="y")
fig.add_scatter3d(mode="markers", marker=dict(size=3))
fig.add_scatter3d(mode="lines")
fig.add_cone(sizemode="absolute", sizeref=2, anchor="tail")

n = 100
n_hoop = 20
t_hoop = np.linspace(0, 2 * np.pi, n_hoop)
x_hoops = HOOP_RADIUS * np.cos(t_hoop)
y_hoops = HOOP_RADIUS * np.sin(t_hoop) + BASKET_Y
z_hoops = np.ones(n_hoop) * BASKET_Z
fig.add_scatter3d(
    x=x_hoops, z=z_hoops, y=y_hoops, mode="lines", line=dict(color="darkred", width=4)
)

def trajectory(t: np.array, vel: np.array) -> np.array:
    """Evolve particles with initial velocities over time, as ballistic trajectory"""
    p = vel * t
    p[:, 2] = p[:, 2] - 0.5 * G * t[:, 0]**2
    return p

def update_graph(points, normals, end_idx, zi, norm):
    fig.data[3].visible = norm
    end_point = np.array([points[end_idx]])
    end_norm = np.array([normals[end_idx]])
    vel, tf = compute_velocities(end_point, zi)
    t = np.linspace(0, tf, 100)
    tr1 = trajectory(t, vel)
    vel[:, 2] = vel[:, 2] - G * tf

    colvel = resolve_collison(vel, end_norm)
    d, thit = hoop_distance(end_point, colvel)
    t2 = np.linspace(0, thit + 0.2, 100)
    tr2 = trajectory(t2, colvel) + points[end_idx]
    if d < HOOP_RADIUS:
        fig.data[0].color = "green"
    else:
        fig.data[0].color = "grey"

    fig.data[0].x = points[:, 0]
    fig.data[0].y = points[:, 1]
    fig.data[0].z = points[:, 2]
    fig.data[3].x = points[:, 0]
    fig.data[3].y = points[:, 1]
    fig.data[3].z = points[:, 2]

    fig.data[3].u = normals[:, 0]
    fig.data[3].v = normals[:, 1]
    fig.data[3].w = normals[:, 2]

    fig.data[1].x = end_point[:, 0]
    fig.data[1].y = end_point[:, 1]
    fig.data[1].z = end_point[:, 2]

    tr = np.concatenate([tr1, tr2])
    fig.data[2].x = tr[:, 0]
    fig.data[2].y = tr[:, 1]
    fig.data[2].z = tr[:, 2]


mtx = threading.Lock()
best_out = w.Output()
x0 = np.zeros(MAX_X_POW * MAX_Y_POW)

# Best solution, 76% accuracy averaged over each possible shot
x0 =  np.array([-0.27230294,  0.2536472,  -0.15352365,  0.39719834, -0.21123466, -0.64555711,
 -0.42602029,  0.82915324, -1.03904597,  0.39507122,  0.20498687,  0.33601243,
  0.49937163, -0.72126431, -0.05917666, -0.02644336,  0.18334707, -1.12240777,
 -0.27573323,  1.72173443, -0.04591846, -0.76348281,  1.30966588, -0.87490057,
  0.24883279, -0.80738162,  0.11858146,  0.62784409, -0.66469961, -0.48329593,
  1.06236067,  0.53730066, -0.05895237,  -0.54345133,  0.01092445])

best_so_far = x0


def callback(xk, convergence=False):
    """Every time a better solution is found update the graph"""
    global best_so_far
    if (xk != best_so_far).any():
        with best_out:
            print("BEST", repr(xk), "loss:", compute_baskets(xk))
        np.savez("best_backboard", xk)
        mtx.acquire()
        best_so_far = xk
        mtx.release()
        refresh()
        interact_fn(4, 1, 1, False)


def run():
    """Takes around 30mins to reach 75% solution"""
    differential_evolution(
        compute_baskets,
        x0=x0,
        bounds=[(-2, 2) for i in x0],
        callback=callback,
        maxiter=int(1e9),
        workers=10,
        updating='deferred'
    )


def refresh():
    global backboard_points
    global backboard_normals
    backboard_points, backboard_normals = setup_backboard(best_so_far, initial_backboard_points)


refresh()


def interact_fn(i, x, y, norm):
    update_graph(backboard_points, backboard_normals, N_GRID_POINTS * x + y, i, norm)


zw = w.interactive(
    interact_fn,
    i=w.IntSlider(value=0, min=0, max=N_Z_VEL),
    x=w.IntSlider(value=N_GRID_POINTS // 2, min=0, max=N_GRID_POINTS - 1),
    y=w.IntSlider(value=N_GRID_POINTS // 2, min=0, max=N_GRID_POINTS - 1),
    norm=False,
)

w = w.VBox([fig, zw, best_out])
t = threading.Thread(target=run)
t.start()
w

    'data': [{'delaunayaxis': 'y', 'opacity': 1, 'type': 'mesh3d', 'uid': '55579…

BEST array([-0.27230294,  0.2536472 , -0.15352365,  0.39719834, -0.21123466,
       -0.64555711, -0.42602029,  0.82915324, -1.03904597,  0.39507122,
        0.20498687,  0.33601243,  0.49937163, -0.72126431, -0.05917666,
       -0.02644336,  0.18334707, -1.12240777, -0.27573323,  1.72173443,
       -0.04591846, -0.76348281,  1.30966588, -0.87490057,  0.24883279,
       -0.80738162,  0.11858146,  0.62784409, -0.66469961, -0.48329593,
        1.06236067,  0.53730066, -0.05895237, -0.54345133,  0.01092445]) loss: -0.7617535903250189
